# Separate sybils in groups

Objetive: based on bot_or_not bot examples, manually analyze some bot accounts and group them by different behavior


In [ ]:
import polars as pl
import os
import json

In [ ]:
DATA_PATH= os.getenv("DATA_PATH", "")

FNAMES_PATH = f"{DATA_PATH}/raw/farcaster-fnames-0-1730134800.parquet"
BOT_OR_NOT_PATH = f"{DATA_PATH}/../farcaster-social-graph-api/farcaster_social_graph_api/persisted_data/bot_or_not_without_info.parquet"
GITCOIN_STAMPS_PATH=F"{DATA_PATH}/interim/final_result_scarsybils_bots.json"


### Load data 

In [ ]:
fnames = pl.read_parquet(FNAMES_PATH)
bot_or_not = pl.read_parquet(BOT_OR_NOT_PATH)
with open(GITCOIN_STAMPS_PATH, 'r') as file:
    gitcoin_stamps = json.load(file)

### Merge bot_or_not with fnames

In [ ]:
bot_or_not  = pl.read_parquet(f'{DATA_PATH}/interim/bot_or_not_without_info.parquet')
bot_or_not

In [ ]:
fnames  = pl.read_parquet(f'{DATA_PATH}/raw/farcaster-fnames-0-1730134800.parquet')
# fnames does not have unique fids
# This happens because it stores all name updates for each fid
# We need to use only the last registered name of each user
last_fnames = fnames[["fid","updated_at"]].group_by("fid").max()
last_fnames = last_fnames.join(fnames,on=["fid","updated_at"],how="left",coalesce=True)[["fid","fname"]]

df = bot_or_not.join(last_fnames[["fid","fname"]],on="fid",how="left", coalesce=True)
df

First group to take a look

In [ ]:
df.filter(pl.col("bot").is_in([True])).sample(n=5,seed=1)

Let's see if we can find some bots inside the human group

In [ ]:
df.filter(pl.col("bot").is_in([False])).sample(n=10,seed=1)

### 1. Declared bots

- Usually have "bot" in their name or profile description
- post consistently about something clearly scheduled

Account examples:
- nasir71
- reaction


### 2. Reward spammers

- Consistently post about "airdrop", "money", "rewards", "claim", "moxie", etc.
- In most part of casts they write @, $ or /link
- likes and recasts are 99% from other bots
- They commonly recast stuff about NFTs, art, drops, etc.

Account examples:
- rovik
- i3
- thunder88


### 3. Masked reward spammers

- just like reward spammers, but they post a lot of consistent generic stuff too
- Consistent generic stuff = "good morning", just a picture cast, random news, random generic opinion about random topic / picture

Account examples:
- rezarashidi
- ali-je01
- faheemmalik
- vlainglong
- hk-habibur

### 4. Random reward spammers
- just like masked reward spammers, but they post a lot of random stuff too

- Account examples:
- bape-
- rezva

### 5. Possible human-controlled sybils
- they seem to be actually controlled by humans, but only for reward collection purpose, with posts that are very feasibly posted by humans
- commonly followed, liked and recasted by bots

Account examples:
- siamsayed